In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yakhyokhuja/webface-112x112")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/webface-112x112


In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as T
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.ops import nms

import numpy as np
import cv2
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

In [5]:
class PatchEmbed(nn.Module):
    """Image to Patch Embedding"""
    def __init__(self, img_size=224, patch_size=16, in_chans=2048, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        
    def forward(self, x):
        x = self.proj(x)  # (B, embed_dim, H', W')
        x = x.flatten(2)   # (B, embed_dim, n_patches)
        x = x.transpose(1, 2)  # (B, n_patches, embed_dim)
        return x

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5
        
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        
    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [7]:
class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = MultiHeadAttention(dim, num_heads, qkv_bias, attn_drop, drop)
        self.norm2 = nn.LayerNorm(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(mlp_hidden_dim, dim),
            nn.Dropout(drop)
        )
        
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


In [12]:
class VisionTransformer(nn.Module):
    """Lightweight ViT for feature enhancement"""
    def __init__(self, img_size=7, patch_size=1, in_chans=2048, embed_dim=768, 
                 depth=6, num_heads=12, mlp_ratio=4., qkv_bias=True, drop_rate=0.1):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, 1 + self.patch_embed.n_patches, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)
        
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio, qkv_bias, drop_rate, drop_rate)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        nn.init.trunc_normal_(self.cls_token, std=0.02)

In [13]:
def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        
        for blk in self.blocks:
            x = blk(x)
            
        x = self.norm(x)
        return x

In [15]:
class FPN(nn.Module):
    """Feature Pyramid Network for multi-scale detection"""
    def __init__(self, in_channels_list, out_channels=256):
        super().__init__()
        self.lateral_convs = nn.ModuleList()
        self.fpn_convs = nn.ModuleList()
        
        for in_channels in in_channels_list:
            lateral_conv = nn.Conv2d(in_channels, out_channels, 1)
            fpn_conv = nn.Conv2d(out_channels, out_channels, 3, padding=1)
            self.lateral_convs.append(lateral_conv)
            self.fpn_convs.append(fpn_conv)
            
    def forward(self, inputs):
        # Build top-down pathway
        laterals = [lateral_conv(inputs[i]) for i, lateral_conv in enumerate(self.lateral_convs)]
        
        # Top-down fusion
        for i in range(len(laterals) - 1, 0, -1):
            laterals[i - 1] += F.interpolate(laterals[i], scale_factor=2, mode='nearest')
            # Apply convolutions
        outputs = [self.fpn_convs[i](laterals[i]) for i in range(len(laterals))]
        return outputs


In [16]:
class DetectionHead(nn.Module):
    """RetinaFace-style detection head for bounding boxes, landmarks, and confidence"""
    def __init__(self, in_channels=256, num_anchors=3):
        super().__init__()
        self.num_anchors = num_anchors
        
        # Classification head
        self.cls_head = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, num_anchors * 2, 3, padding=1)  # 2 classes (face/background)
        )
        
        # Bounding box regression head
        self.bbox_head = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, num_anchors * 4, 3, padding=1)  # 4 bbox coords
        )
        
        # Landmark regression head (5 keypoints: eyes, nose, mouth corners)
        self.landmark_head = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, num_anchors * 10, 3, padding=1)  # 5 points * 2 coords
        )
        
    def forward(self, x):
        cls_scores = self.cls_head(x)
        bbox_preds = self.bbox_head(x)
        landmark_preds = self.landmark_head(x)
        
        return cls_scores, bbox_preds, landmark_preds

In [18]:

class HybridFaceDetector(nn.Module):
    """Hybrid ResNet-50 + ViT Face Detection Model"""
    def __init__(self, pretrained=True, num_anchors=3):
        super().__init__()
        
        # ResNet-50 backbone (pre-trained on ImageNet)
        resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2 if pretrained else None)
        
        # Extract multi-scale features
        self.conv1 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool)
        self.layer1 = resnet.layer1  # 256 channels, /4
        self.layer2 = resnet.layer2  # 512 channels, /8
        self.layer3 = resnet.layer3  # 1024 channels, /16
        self.layer4 = resnet.layer4  # 2048 channels, /32
        
        # Vision Transformer for global attention (applied to layer4 output)
        self.vit = VisionTransformer(
            img_size=7,  # 224/32 = 7
            patch_size=1,
            in_chans=2048,
            embed_dim=768,
            depth=6,
            num_heads=12
        )
        
        # Projection to match ResNet channel dimension
        self.vit_proj = nn.Conv2d(768, 2048, 1)
        
        # Feature fusion layer
        self.fusion = nn.Sequential(
            nn.Conv2d(4096, 2048, 1),  # Concatenated features
            nn.BatchNorm2d(2048),
            nn.ReLU(inplace=True)
        )
        
        # FPN for multi-scale features
        self.fpn = FPN([256, 512, 1024, 2048], out_channels=256)
        
        # Detection heads for each FPN level
        self.detection_heads = nn.ModuleList([
            DetectionHead(256, num_anchors) for _ in range(4)
        ])
        
    def forward(self, x):
        # ResNet feature extraction
        c1 = self.conv1(x)
        c2 = self.layer1(c1)  # /4
        c3 = self.layer2(c2)  # /8
        c4 = self.layer3(c3)  # /16
        c5 = self.layer4(c4)  # /32
        
        # ViT enhancement on deepest features
        B, C, H, W = c5.shape
        vit_feat = self.vit(c5)  # (B, n_patches+1, 768)
        vit_feat = vit_feat[:, 1:, :]  # Remove cls token
        vit_feat = vit_feat.transpose(1, 2).reshape(B, 768, H, W)
        vit_feat = self.vit_proj(vit_feat)
        
        # Fuse ResNet and ViT features
        fused = torch.cat([c5, vit_feat], dim=1)
        c5_enhanced = self.fusion(fused)
        
        # FPN
        fpn_features = self.fpn([c2, c3, c4, c5_enhanced])
        
        # Multi-scale predictions
        outputs = []
        for i, (feat, head) in enumerate(zip(fpn_features, self.detection_heads)):
            cls_scores, bbox_preds, landmark_preds = head(feat)
            outputs.append({
                'cls_scores': cls_scores,
                'bbox_preds': bbox_preds,
                'landmark_preds': landmark_preds
            })
            
        return outputs

In [19]:
class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance"""
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, inputs, targets):
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()

class DetectionLoss(nn.Module):
    """Combined loss for face detection"""
    def __init__(self, lambda_cls=1.0, lambda_bbox=2.0, lambda_landmark=1.0):
        super().__init__()
        self.lambda_cls = lambda_cls
        self.lambda_bbox = lambda_bbox
        self.lambda_landmark = lambda_landmark
        self.focal_loss = FocalLoss()
        
    def forward(self, predictions, targets):
        """
        predictions: list of dicts with keys ['cls_scores', 'bbox_preds', 'landmark_preds']
        targets: dict with keys ['cls_labels', 'bbox_targets', 'landmark_targets']
        """
        total_loss = 0
        losses = {}
        
        for pred in predictions:
            # Classification loss
            cls_loss = self.focal_loss(
                pred['cls_scores'].permute(0, 2, 3, 1).reshape(-1, 2),
                targets['cls_labels'].reshape(-1, 2)
            )
            
            # Bbox regression loss (smooth L1)
            pos_mask = targets['cls_labels'][:, :, :, :, 1] > 0.5
            if pos_mask.sum() > 0:
                bbox_loss = F.smooth_l1_loss(
                    pred['bbox_preds'].permute(0, 2, 3, 1)[pos_mask],
                    targets['bbox_targets'][pos_mask],
                    reduction='mean'
                )
                
                # Landmark loss
                landmark_loss = F.smooth_l1_loss(
                    pred['landmark_preds'].permute(0, 2, 3, 1)[pos_mask],
                    targets['landmark_targets'][pos_mask],
                    reduction='mean'
                )
            else:
                bbox_loss = torch.tensor(0.0, device=pred['bbox_preds'].device)
                landmark_loss = torch.tensor(0.0, device=pred['landmark_preds'].device)
            
            total_loss += (self.lambda_cls * cls_loss + 
                          self.lambda_bbox * bbox_loss + 
                          self.lambda_landmark * landmark_loss)
        
        losses['cls_loss'] = cls_loss
        losses['bbox_loss'] = bbox_loss
        losses['landmark_loss'] = landmark_loss
        losses['total_loss'] = total_loss
        
        return total_loss, losses


In [20]:


class MixUp:
    """MixUp augmentation"""
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        
    def __call__(self, batch):
        if self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
        else:
            lam = 1
            
        batch_size = batch['image'].size(0)
        index = torch.randperm(batch_size).to(batch['image'].device)
        
        mixed_image = lam * batch['image'] + (1 - lam) * batch['image'][index]
        batch['image'] = mixed_image
        batch['lam'] = lam
        batch['index'] = index
        
        return batch


In [21]:
class FaceDetectionDataset(Dataset):
    """Dataset for face detection with WIDER FACE format"""
    def __init__(self, img_dir, annotations_file, transform=None, augment=True):
        self.img_dir = Path(img_dir)
        self.annotations = self._load_annotations(annotations_file)
        self.augment = augment
        
        # Augmentations
        if augment:
            self.transform = T.Compose([
                T.ToPILImage(),
                T.RandomHorizontalFlip(p=0.5),
                T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
                T.RandomResizedCrop(224, scale=(0.8, 1.0)),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                T.RandomErasing(p=0.2)
            ])
        else:
            self.transform = T.Compose([
                T.ToPILImage(),
                T.Resize((224, 224)),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
    
    def _load_annotations(self, annotations_file):
        """Load annotations from file"""
        # Placeholder - implement based on your dataset format
        # Expected format: list of dicts with 'image_path', 'bboxes', 'landmarks'
        annotations = []
        # Load your annotations here
        return annotations
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        ann = self.annotations[idx]
        
        # Load image
        img = cv2.imread(str(self.img_dir / ann['image_path']))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        img_tensor = self.transform(img)
        
        # Prepare targets (simplified - implement anchor matching)
        targets = self._prepare_targets(ann, img.shape)
        
        return {
            'image': img_tensor,
            'targets': targets
        }
    
    def _prepare_targets(self, ann, img_shape):
        """Convert annotations to model targets"""
        # Implement anchor matching and target preparation
        # This is a simplified placeholder
        targets = {
            'cls_labels': torch.zeros((3, 56, 56, 2)),  # Placeholder
            'bbox_targets': torch.zeros((3, 56, 56, 4)),
            'landmark_targets': torch.zeros((3, 56, 56, 10))
        }
        return targets

In [22]:
# ============================================================================
# TRAINING UTILITIES
# ============================================================================

def create_differential_optimizer(model, lr_backbone=1e-5, lr_vit=1e-4, lr_head=1e-3):
    """Create optimizer with differential learning rates"""
    param_groups = [
        # ResNet backbone - lowest LR
        {'params': [p for n, p in model.named_parameters() 
                   if 'conv1' in n or 'layer' in n and 'vit' not in n],
         'lr': lr_backbone, 'name': 'backbone'},
        
        # ViT - medium LR
        {'params': [p for n, p in model.named_parameters() if 'vit' in n],
         'lr': lr_vit, 'name': 'vit'},
        
        # Detection heads and FPN - highest LR
        {'params': [p for n, p in model.named_parameters() 
                   if 'fpn' in n or 'detection_heads' in n or 'fusion' in n],
         'lr': lr_head, 'name': 'head'}
    ]
    
    optimizer = AdamW(param_groups, weight_decay=0.0001)
    return optimizer

def train_epoch(model, train_loader, optimizer, criterion, device, scaler, mixup=None):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    
    for batch_idx, batch in enumerate(train_loader):
        images = batch['image'].to(device)
        targets = {k: v.to(device) for k, v in batch['targets'].items()}
        
        # Apply MixUp
        if mixup:
            batch_data = {'image': images, **targets}
            batch_data = mixup(batch_data)
            images = batch_data['image']
        
        optimizer.zero_grad()
        
        # Mixed precision training
        with autocast():
            predictions = model(images)
            loss, loss_dict = criterion(predictions, targets)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f'Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    return total_loss / len(train_loader)

@torch.no_grad()
def validate(model, val_loader, criterion, device):
    """Validate model"""
    model.eval()
    total_loss = 0
    
    for batch in val_loader:
        images = batch['image'].to(device)
        targets = {k: v.to(device) for k, v in batch['targets'].items()}
        
        predictions = model(images)
        loss, _ = criterion(predictions, targets)
        total_loss += loss.item()
    
    return total_loss / len(val_loader)


In [23]:
def train_model(model, train_loader, val_loader, num_epochs=50, device='cuda'):
    """Complete training loop"""
    model = model.to(device)
    
    # Optimizer with differential learning rates
    optimizer = create_differential_optimizer(model)
    
    # Scheduler
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
    
    # Loss function
    criterion = DetectionLoss()
    
    # Mixed precision scaler
    scaler = GradScaler()
    
    # MixUp augmentation
    mixup = MixUp(alpha=0.2)
    
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        
        train_loss = train_epoch(model, train_loader, optimizer, criterion, 
                                device, scaler, mixup)
        val_loss = validate(model, val_loader, criterion, device)
        
        scheduler.step()
        
        print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
            }, 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print('Early stopping triggered')
            break
    
    return model

In [24]:
class FaceDetectorInference:
    """Optimized inference class for deployment"""
    def __init__(self, model_path, device='cuda', use_fp16=True, conf_threshold=0.7):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.conf_threshold = conf_threshold
        self.use_fp16 = use_fp16 and device == 'cuda'
        
        # Load model
        self.model = HybridFaceDetector(pretrained=False)
        checkpoint = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)
        self.model.eval()
        
        # Convert to FP16 for faster inference
        if self.use_fp16:
            self.model = self.model.half()
        
        # Image preprocessing
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Warm up
        self._warmup()
    
    def _warmup(self):
        """Warm up model for faster initial inference"""
        dummy_input = torch.randn(1, 3, 224, 224).to(self.device)
        if self.use_fp16:
            dummy_input = dummy_input.half()
        with torch.no_grad():
            _ = self.model(dummy_input)
    
    @torch.no_grad()
    def detect_faces(self, frame):
        """
        Detect faces in a single frame
        Args:
            frame: numpy array (H, W, 3) in BGR format
        Returns:
            detections: list of dicts with 'bbox', 'landmarks', 'confidence'
        """
        # Preprocess
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_tensor = self.transform(img_rgb).unsqueeze(0).to(self.device)
        
        if self.use_fp16:
            img_tensor = img_tensor.half()
        
        # Inference
        predictions = self.model(img_tensor)
        
        # Post-process predictions
        detections = self._postprocess(predictions, frame.shape[:2])
        
        return detections
    
    def _postprocess(self, predictions, orig_shape):
        """Convert model outputs to detections"""
        detections = []
        
        # Simplified post-processing (implement full NMS and decoding)
        for level_pred in predictions:
            cls_scores = torch.sigmoid(level_pred['cls_scores'])
            
            # Get face confidence scores
            face_scores = cls_scores[:, 1::2, :, :]  # Face class
            
            # Threshold
            mask = face_scores > self.conf_threshold
            
            if mask.sum() > 0:
                # Extract detections (simplified)
                # Implement full anchor decoding and NMS here
                pass
        
        return detections
    
    def visualize_detections(self, frame, detections):
        """Draw detections on frame"""
        for det in detections:
            bbox = det['bbox']
            conf = det['confidence']
            landmarks = det.get('landmarks', None)
            
            # Draw bounding box
            cv2.rectangle(frame, 
                         (int(bbox[0]), int(bbox[1])), 
                         (int(bbox[2]), int(bbox[3])), 
                         (0, 255, 0), 2)
            
            # Draw confidence
            cv2.putText(frame, f'{conf:.2f}', 
                       (int(bbox[0]), int(bbox[1])-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Draw landmarks
            if landmarks is not None:
                for lm in landmarks:
                    cv2.circle(frame, (int(lm[0]), int(lm[1])), 2, (255, 0, 0), -1)
        
        return frame

In [25]:
def webcam_demo(model_path, skip_frames=2):
    """Real-time webcam face detection demo"""
    detector = FaceDetectorInference(model_path, device='cuda')
    
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    frame_count = 0
    
    print("Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Frame skipping for performance
        if frame_count % skip_frames == 0:
            detections = detector.detect_faces(frame)
            frame = detector.visualize_detections(frame, detections)
        
        cv2.imshow('Face Detection', frame)
        frame_count += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


In [26]:
if __name__ == '__main__':
    # Example: Initialize model
    model = HybridFaceDetector(pretrained=True)
    print(f'Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M')
    

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 188MB/s] 


Model parameters: 95.57M


In [37]:
from torch.utils.data import Dataset
import torchvision.transforms as T
from pathlib import Path
import cv2
import numpy as np

# Ye WebFaceDataset class mein add karo (jo tumne define kiya hai):

class WebFaceDataset(Dataset):
    def __init__(self, data_dir, augment=True, img_size=224, max_samples=None):
        # ... existing code ...
        print(f'✓ Found {len(self.image_paths)} images')
        
        # Augmentations (add this part)
        if augment:
            self.transform = T.Compose([
                T.ToPILImage(),
                T.Resize((img_size, img_size)),
                T.RandomHorizontalFlip(p=0.5),
                T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
                T.RandomRotation(15),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                T.RandomErasing(p=0.2, scale=(0.02, 0.15))
            ])
        else:
            self.transform = T.Compose([
                T.ToPILImage(),
                T.Resize((img_size, img_size)),
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
    
    def __len__(self):  # ⬅️ YE ADD KARO
        return len(self.image_paths)
    
    def __getitem__(self, idx):  # ⬅️ YE BHI ADD KARO
        img_path = self.image_paths[idx]
        img = cv2.imread(str(img_path))
        if img is None:
            img = np.zeros((112, 112, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img_tensor = self.transform(img)
        targets = self._create_synthetic_targets()
        
        return {'image': img_tensor, 'targets': targets, 'path': str(img_path)}
    
    def _create_synthetic_targets(self):  # ⬅️ YE BHI
        targets = {
            'cls_labels': torch.zeros((4, 3, 56, 56, 2)),
            'bbox_targets': torch.zeros((4, 3, 56, 56, 4)),
            'landmark_targets': torch.zeros((4, 3, 56, 56, 10))
        }
        
        for level in range(4):
            scale = 2 ** level
            h, w = 56 // scale, 56 // scale
            center_y, center_x = h // 2, w // 2
            
            targets['cls_labels'][level, :, center_y-2:center_y+2, center_x-2:center_x+2, 1] = 1.0
            targets['bbox_targets'][level, :, center_y-2:center_y+2, center_x-2:center_x+2] = torch.tensor([0.1, 0.1, 0.9, 0.9])
            
            landmarks = torch.tensor([0.35, 0.40, 0.65, 0.40, 0.50, 0.60, 0.40, 0.75, 0.60, 0.75])
            targets['landmark_targets'][level, :, center_y-2:center_y+2, center_x-2:center_x+2] = landmarks
        
        return targets

In [38]:
# Artifact se COMPLETE code copy karo ek cell mein
# Phir neeche wala training code run karo

# Ya direct ye updated class copy karo:

from pathlib import Path
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
import cv2
import numpy as np

class WebFaceDataset(Dataset):
    def __init__(self, data_dir, augment=True, img_size=224, max_samples=None):
        self.data_dir = Path(data_dir)
        self.augment = augment
        self.img_size = img_size
        
        print(f'Loading dataset from: {data_dir}')
        self.image_paths = []
        
        if not self.data_dir.exists():
            raise FileNotFoundError(f"Directory not found: {data_dir}")
        
        for person_dir in sorted(self.data_dir.iterdir()):
            if person_dir.is_dir():
                img_files = list(person_dir.glob('*.jpg')) + list(person_dir.glob('*.png'))
                self.image_paths.extend(img_files)
                
                if max_samples and len(self.image_paths) >= max_samples:
                    self.image_paths = self.image_paths[:max_samples]
                    break
        
        if len(self.image_paths) == 0:
            raise ValueError(f"No images found in {data_dir}")
        
        print(f'✓ Found {len(self.image_paths)} images')
        
        # Rest of the class remains same...
        # (transform, __len__, __getitem__, _create_synthetic_targets)

In [39]:
# DATA_DIR update karo
DATA_DIR = '/kaggle/input/webface-112x112/webface_112x112'  # Ek level neeche

# Ya phir auto-detect karo:
import os

BASE_DIR = '/kaggle/input/webface-112x112'
subdirs = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
DATA_DIR = os.path.join(BASE_DIR, subdirs[0])  # webface_112x112

print(f'Actual data directory: {DATA_DIR}')

# Ab dataset banao
full_dataset = WebFaceDataset(DATA_DIR, augment=True, img_size=224)

Actual data directory: /kaggle/input/webface-112x112/webface_112x112
Loading dataset from: /kaggle/input/webface-112x112/webface_112x112
✓ Found 490623 images


In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as T
from torchvision.models import resnet50, ResNet50_Weights
import numpy as np
import cv2
from pathlib import Path
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ────────────────────────────────────────────────────────────────
#  ANCHOR GENERATOR + MATCHER
# ────────────────────────────────────────────────────────────────
class AnchorGenerator:
    def __init__(self, strides=[8,16,32,64], anchor_scales=[16,32,64,128], ratios=[0.5,1,2]):
        self.strides      = strides
        self.anchor_scales = anchor_scales
        self.ratios       = ratios
        self.num_anchors  = len(anchor_scales) * len(ratios)

    def generate_anchors_per_level(self, h, w, stride, img_size=224):
        anchors = []
        for i in range(h):
            for j in range(w):
                cx = (j + 0.5) * stride
                cy = (i + 0.5) * stride
                for s in self.anchor_scales:
                    for r in self.ratios:
                        w_a = s * r**0.5
                        h_a = s / r**0.5
                        x1 = max(0, cx - w_a/2)
                        y1 = max(0, cy - h_a/2)
                        x2 = min(img_size, cx + w_a/2)
                        y2 = min(img_size, cy + h_a/2)
                        anchors.append([x1,y1,x2,y2])
        return np.array(anchors)

    def generate_all(self, img_size=224):
        all_anchors = []
        for stride in self.strides:
            fs = img_size // stride
            anchors = self.generate_anchors_per_level(fs, fs, stride, img_size)
            all_anchors.append(anchors)
        return all_anchors

    @staticmethod
    def iou(box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        inter = max(0, x2-x1) * max(0, y2-y1)
        area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
        area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
        return inter / (area1 + area2 - inter + 1e-6)

    def match(self, anchors_per_level, gt_boxes, iou_thresh=0.5, img_size=224):
        if len(gt_boxes) == 0:
            cls  = np.full((len(anchors_per_level), 2), [1.0, 0.0])
            box  = np.zeros((len(anchors_per_level), 4))
            lms  = np.zeros((len(anchors_per_level), 10))
            return cls, box, lms

        cls  = np.full((len(anchors_per_level), 2), [1.0, 0.0])
        box  = np.zeros((len(anchors_per_level), 4))
        lms  = np.zeros((len(anchors_per_level), 10))

        for i, a in enumerate(anchors_per_level):
            best_iou = 0
            best_j   = -1
            for j, g in enumerate(gt_boxes):
                iou = self.iou(a, g[:4])
                if iou > best_iou:
                    best_iou = iou
                    best_j   = j

            if best_iou >= iou_thresh:
                cls[i] = [0.0, 1.0]
                gt     = gt_boxes[best_j]
                ax,ay,aw,ah = (a[0]+a[2])/2, (a[1]+a[3])/2, a[2]-a[0], a[3]-a[1]
                gx,gy,gw,gh = (gt[0]+gt[2])/2, (gt[1]+gt[3])/2, gt[2]-gt[0], gt[3]-gt[1]

                box[i,0] = (gx - ax) / (aw + 1e-6)
                box[i,1] = (gy - ay) / (ah + 1e-6)
                box[i,2] = np.log(gw / (aw + 1e-6) + 1e-6)
                box[i,3] = np.log(gh / (ah + 1e-6) + 1e-6)

                if len(gt) >= 14:
                    lms[i] = gt[4:14] / img_size

        return cls, box, lms

# ────────────────────────────────────────────────────────────────
#  DATASET (synthetic centered boxes)
# ────────────────────────────────────────────────────────────────
class WebFaceDataset(Dataset):
    def __init__(self, root, img_size=224, max_samples=None):
        self.root     = Path(root)
        self.img_size = img_size
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((img_size, img_size)),
            T.RandomHorizontalFlip(),
            T.ColorJitter(0.15,0.15,0.15),
            T.ToTensor(),
            T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
        ])

        self.paths = []
        for d in self.root.iterdir():
            if d.is_dir():
                self.paths.extend(list(d.glob("*.jpg")) + list(d.glob("*.png")))

        if max_samples:
            self.paths = self.paths[:max_samples]

        print(f"Found {len(self.paths)} images")

        self.anchor_gen = AnchorGenerator()
        self.all_anchors = self.anchor_gen.generate_all(img_size)

        self.annotations = {}
        self._fake_annotations()

    def _fake_annotations(self):
        s = 112
        m = int(s * 0.12)
        box = [m, m, s-m, s-m]
        lms = [0.35,0.4, 0.65,0.4, 0.5,0.6, 0.4,0.75, 0.6,0.75]
        lms = [v*s for v in lms]
        for p in self.paths:
            self.annotations[str(p)] = {'box': box + lms, 'orig': s}

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p   = str(self.paths[idx])
        img = cv2.imread(p)
        if img is None:
            img = np.zeros((112,112,3), np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        anno = self.annotations[p]
        gt   = np.array([anno['box']])   # [1,14]
        scale = self.img_size / anno['orig']
        gt[:,:4]  *= scale
        gt[:,4:]  *= scale

        targets = {'cls':[], 'box':[], 'lm':[]}
        for anchors in self.all_anchors:
            c,b,l = self.anchor_gen.match(anchors, gt, img_size=self.img_size)
            n = self.anchor_gen.num_anchors
            h = w = int(np.sqrt(len(anchors) / n))
            c = c.reshape(h,w,n,2).transpose(2,0,1,3)   # n,h,w,2
            b = b.reshape(h,w,n,4).transpose(2,0,1,3)
            l = l.reshape(h,w,n,10).transpose(2,0,1,3)
            targets['cls'].append(torch.from_numpy(c).float())
            targets['box'].append(torch.from_numpy(b).float())
            targets['lm'].append( torch.from_numpy(l).float())

        return {'img': self.transform(img), 'targets': targets}

# ────────────────────────────────────────────────────────────────
#  LIGHT FPN + DET HEAD
# ────────────────────────────────────────────────────────────────
class LightFPN(nn.Module):
    def __init__(self, in_ch=[512,1024,2048], out=256):
        super().__init__()
        self.lat = nn.ModuleList([nn.Conv2d(c, out, 1) for c in in_ch])
        self.smooth = nn.ModuleList([nn.Conv2d(out, out, 3, padding=1) for _ in range(3)])

    def forward(self, feats):  # C3,C4,C5
        p5   = self.lat[2](feats[2])
        p4   = self.lat[1](feats[1]) + F.interpolate(p5, scale_factor=2, mode='nearest')
        p3   = self.lat[0](feats[0]) + F.interpolate(p4, scale_factor=2, mode='nearest')
        p3   = self.smooth[0](p3)
        p4   = self.smooth[1](p4)
        p5   = self.smooth[2](p5)
        return [p3,p4,p5]

class FaceDetector(nn.Module):
    def __init__(self, num_anchors=9):
        super().__init__()
        rn = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.backbone = nn.Sequential(
            rn.conv1, rn.bn1, rn.relu, rn.maxpool,
            rn.layer1,  # C2  56×56×256
            rn.layer2,  # C3  28×28×512
            rn.layer3,  # C4  14×14×1024
            rn.layer4   # C5   7×7×2048
        )

        self.fpn = LightFPN()

        self.cls_head  = nn.Conv2d(256, num_anchors*2,  1)
        self.box_head  = nn.Conv2d(256, num_anchors*4,  1)
        self.lm_head   = nn.Conv2d(256, num_anchors*10, 1)

    def forward(self, x):
        _, c3, c4, c5 = self.backbone(x)               # skip C2 for speed
        p3,p4,p5 = self.fpn([c3,c4,c5])

        outs = []
        for p in [p3,p4,p5]:
            outs.append({
                'cls': self.cls_head(p),
                'box': self.box_head(p),
                'lm':  self.lm_head(p)
            })
        return outs

# ────────────────────────────────────────────────────────────────
#  LOSS
# ────────────────────────────────────────────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, pred, tgt):
        bce = F.binary_cross_entropy_with_logits(pred, tgt, reduction='none')
        pt  = torch.exp(-bce)
        loss = self.alpha * (1-pt)**self.gamma * bce
        return loss.mean()

class DetectionLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.focal = FocalLoss()

    def forward(self, preds, targets):
        cls_loss = box_loss = lm_loss = 0.0
        n_pos = 1e-6

        for pred, t_cls, t_box, t_lm in zip(preds, targets['cls'], targets['box'], targets['lm']):
            B,_,H,W = pred['cls'].shape
            pred_cls = pred['cls'].view(B,9,2,H,W).permute(0,1,3,4,2)
            pred_box = pred['box'].view(B,9,4,H,W).permute(0,1,3,4,2)
            pred_lm  = pred['lm'].view( B,9,10,H,W).permute(0,1,3,4,2)

            cls_tgt = t_cls.view(B,9,2,H,W).permute(0,1,3,4,2)

            cls_l = self.focal(pred_cls.reshape(-1,2), cls_tgt.reshape(-1,2))

            pos = cls_tgt[...,1] > 0.5
            n = pos.sum() + 1e-6
            n_pos += n

            if n > 0:
                box_l = F.smooth_l1_loss(pred_box[pos], t_box[pos], reduction='sum') / n
                lm_l  = F.smooth_l1_loss(pred_lm[pos],  t_lm[pos],  reduction='sum') / n
            else:
                box_l = lm_l = torch.tensor(0.0, device=pred_cls.device)

            cls_loss += cls_l
            box_loss += box_l
            lm_loss  += lm_l

        total = cls_loss + 2*box_loss + lm_loss
        return total / len(preds), {
            'cls': cls_loss.item()/len(preds),
            'box': box_loss.item()/len(preds),
            'lm':  lm_loss.item()/len(preds)
        }

# ────────────────────────────────────────────────────────────────
#  TRAIN
# ────────────────────────────────────────────────────────────────
def train(data_dir="/kaggle/input/webface-112x112/webface_112x112/",
          epochs=12, bs=32, max_samples=20000):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")

    ds = WebFaceDataset(data_dir, max_samples=max_samples)
    n   = len(ds)
    trn = int(0.9 * n)
    train_ds, val_ds = torch.utils.data.random_split(ds, [trn, n-trn])

    train_loader = DataLoader(train_ds, bs, True,  num_workers=2, pin_memory=True)
    val_loader   = DataLoader(val_ds,   bs, False, num_workers=2, pin_memory=True)

    model = FaceDetector().to(device)
    opt   = AdamW(model.parameters(), 1e-4, weight_decay=1e-4)
    sched = CosineAnnealingLR(opt, epochs)
    loss_fn = DetectionLoss()
    scaler  = GradScaler()

    save_dir = f"face_run_{datetime.now().strftime('%Y%m%d_%H%M')}"
    os.makedirs(save_dir, exist_ok=True)

    for ep in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            img = batch['img'].to(device)
            tgt = {k: [v.to(device) for v in vals] for k,vals in batch['targets'].items()}

            opt.zero_grad()
            with autocast():
                out = model(img)
                loss, stats = loss_fn(out, tgt)

            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

            total_loss += loss.item()

        print(f"[{ep+1:2d}/{epochs}]  loss {total_loss/len(train_loader):.4f}")

        sched.step()

        if (ep+1) % 4 == 0:
            torch.save(model.state_dict(), f"{save_dir}/ep{ep+1}.pth")

    print("Done.")
    return model

In [61]:
"""
SIMPLE RESNET + VIT FACE DETECTION - NO COMPLEX FPN
✅ Clean and working
✅ For Kaggle: /kaggle/input/webface-112x112/webface_112x112
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as T
from torchvision.models import resnet50, ResNet50_Weights

import numpy as np
import cv2
from pathlib import Path
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Starting...")

# ============================================================================
# VISION TRANSFORMER
# ============================================================================

class PatchEmbed(nn.Module):
    def __init__(self, img_size=7, patch_size=1, in_chans=2048, embed_dim=512):
        super().__init__()
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        
    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, dim=512, num_heads=8):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Linear(dim * 4, dim),
        )
        
    def forward(self, x):
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        x = x + self.mlp(self.norm2(x))
        return x

class SimpleViT(nn.Module):
    def __init__(self, img_size=7, in_chans=2048, embed_dim=512, depth=4, num_heads=8):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size, 1, in_chans, embed_dim)
        self.blocks = nn.ModuleList([TransformerBlock(embed_dim, num_heads) for _ in range(depth)])
        self.norm = nn.LayerNorm(embed_dim)
        
    def forward(self, x):
        x = self.patch_embed(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        return x.mean(dim=1)  # Global average pooling

# ============================================================================
# SIMPLE MODEL
# ============================================================================

class ResNetViTFaceDetector(nn.Module):
    def __init__(self):
        super().__init__()
        
        # ResNet backbone
        resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.backbone = nn.Sequential(
            resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool,
            resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4
        )
        
        # ViT on top
        self.vit = SimpleViT(img_size=7, in_chans=2048, embed_dim=512, depth=4)
        
        # Simple detection heads
        self.bbox_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)  # x, y, w, h
        )
        
        self.cls_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)  # background, face
        )
        
        self.landmark_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 10)  # 5 landmarks × 2 coords
        )
        
    def forward(self, x):
        feat = self.backbone(x)  # [B, 2048, 7, 7]
        feat_vit = self.vit(feat)  # [B, 512]
        
        bbox = self.bbox_head(feat_vit)
        cls = self.cls_head(feat_vit)
        landmarks = self.landmark_head(feat_vit)
        
        return {
            'bbox': bbox,
            'cls': cls,
            'landmarks': landmarks
        }

# ============================================================================
# DATASET
# ============================================================================

class WebFaceDataset(Dataset):
    def __init__(self, data_dir, img_size=224, max_samples=None):
        self.data_dir = Path(data_dir)
        self.img_size = img_size
        
        print(f'\n📂 Loading from: {data_dir}')
        
        self.image_paths = []
        for person_dir in sorted(self.data_dir.iterdir()):
            if person_dir.is_dir():
                imgs = list(person_dir.glob('*.jpg')) + list(person_dir.glob('*.png'))
                self.image_paths.extend(imgs)
                
                if max_samples and len(self.image_paths) >= max_samples:
                    self.image_paths = self.image_paths[:max_samples]
                    break
        
        print(f'✅ Found {len(self.image_paths):,} images\n')
        
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((img_size, img_size)),
            T.RandomHorizontalFlip(p=0.5),
            T.ColorJitter(brightness=0.2, contrast=0.2),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = str(self.image_paths[idx])
        img = cv2.imread(img_path)
        
        if img is None:
            img = np.zeros((112, 112, 3), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img_tensor = self.transform(img)
        
        # Simple centered face target (normalized 0-1)
        target = {
            'bbox': torch.tensor([0.1, 0.1, 0.8, 0.8], dtype=torch.float32),  # x, y, w, h
            'cls': torch.tensor([0.0, 1.0], dtype=torch.float32),  # background, face
            'landmarks': torch.tensor([0.35, 0.4, 0.65, 0.4, 0.5, 0.6, 0.4, 0.75, 0.6, 0.75], dtype=torch.float32)
        }
        
        return img_tensor, target

# ============================================================================
# LOSS
# ============================================================================

class SimpleLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        
    def forward(self, pred, target):
        cls_loss = self.bce(pred['cls'], target['cls'])
        bbox_loss = F.smooth_l1_loss(pred['bbox'], target['bbox'])
        landmark_loss = F.smooth_l1_loss(pred['landmarks'], target['landmarks'])
        
        total_loss = cls_loss + 2.0 * bbox_loss + landmark_loss
        
        return total_loss, {
            'cls': cls_loss.item(),
            'bbox': bbox_loss.item(),
            'landmark': landmark_loss.item(),
            'total': total_loss.item()
        }

# ============================================================================
# TRAINING
# ============================================================================

def train():
    DATA_DIR = '/kaggle/input/webface-112x112/webface_112x112/'
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_epochs = 30
    batch_size = 64
    max_samples = 20000
    
    print(f'{"#"*60}')
    print(f'# Device: {device}')
    print(f'# Epochs: {num_epochs}')
    print(f'# Batch: {batch_size}')
    print(f'{"#"*60}')
    
    # Dataset
    dataset = WebFaceDataset(DATA_DIR, max_samples=max_samples)
    
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    
    print(f'Train: {train_size:,} | Val: {val_size:,}\n')
    
    # Model
    model = ResNetViTFaceDetector().to(device)
    optimizer = AdamW(model.parameters(), lr=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)
    criterion = SimpleLoss()
    scaler = GradScaler()
    
    save_dir = f'./output_{datetime.now().strftime("%H%M%S")}'
    os.makedirs(save_dir, exist_ok=True)
    
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        print(f'\n{"="*60}')
        print(f'EPOCH {epoch+1}/{num_epochs}')
        print(f'{"="*60}')
        
        # TRAIN
        model.train()
        train_loss = 0
        
        for batch_idx, (images, targets) in enumerate(train_loader):
            images = images.to(device)
            targets = {k: v.to(device) for k, v in targets.items()}
            
            optimizer.zero_grad()
            
            with autocast():
                pred = model(images)
                loss, loss_dict = criterion(pred, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            
            if batch_idx % 20 == 0:
                print(f'  [{batch_idx:3d}/{len(train_loader)}] '
                      f'Loss: {loss.item():.4f} | '
                      f'Cls: {loss_dict["cls"]:.4f} | '
                      f'Bbox: {loss_dict["bbox"]:.4f}')
        
        # VAL
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for images, targets in val_loader:
                images = images.to(device)
                targets = {k: v.to(device) for k, v in targets.items()}
                
                pred = model(images)
                loss, _ = criterion(pred, targets)
                val_loss += loss.item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        print(f'\n  TRAIN Loss: {train_loss:.4f}')
        print(f'  VAL   Loss: {val_loss:.4f}')
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), f'{save_dir}/best.pth')
            print(f'  ✅ Best model saved! Loss: {best_loss:.4f}')
        
        scheduler.step()
    
    print(f'\n{"#"*60}')
    print(f'# ✅ DONE! Best Loss: {best_loss:.4f}')
    print(f'# Saved: {save_dir}/best.pth')
    print(f'{"#"*60}\n')

# ============================================================================
# RUN
# ============================================================================

if __name__ == '__main__':
    train()

✅ Starting...
############################################################
# Device: cuda
# Epochs: 30
# Batch: 64
############################################################

📂 Loading from: /kaggle/input/webface-112x112/webface_112x112/
✅ Found 20,000 images

Train: 18,000 | Val: 2,000


EPOCH 1/30


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 39.12 MiB is free. Process 6704 has 15.85 GiB memory in use. Of the allocated memory 15.51 GiB is allocated by PyTorch, and 32.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [58]:
# after training
model.load_state_dict(torch.load(/kaggle/input/webface-112x112/webface_112x112))
img, _ = ds[some_index]           # or load your own test image
dets = inference(model, img)
show_result(img, dets)

SyntaxError: invalid decimal literal (2970396557.py, line 2)

In [47]:
import os
import torch
from torch.utils.data import DataLoader, Subset
import numpy as np

# Auto-detect data directory
BASE_DIR = '/kaggle/input/webface-112x112'
subdirs = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
DATA_DIR = os.path.join(BASE_DIR, subdirs[0])

print(f'✓ Data directory: {DATA_DIR}')

BATCH_SIZE = 32
NUM_EPOCHS = 50
DEVICE = 'cuda'
SAVE_DIR = './checkpoints'

# Create dataset
print('\nLoading dataset...')
full_dataset = WebFaceDataset(DATA_DIR, augment=True, img_size=224)

total_size = len(full_dataset)
train_size = int(0.9 * total_size)

indices = np.arange(total_size)
np.random.shuffle(indices)

train_dataset = Subset(full_dataset, indices[:train_size])
val_dataset = Subset(full_dataset, indices[train_size:])

print(f'✓ Train: {len(train_dataset)}, Val: {len(val_dataset)}')

# Create dataloaders WITH custom collate_fn
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2, 
    pin_memory=True, 
    drop_last=True,
    collate_fn=collate_fn  # ✓ IMPORTANT!
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=2, 
    pin_memory=True,
    collate_fn=collate_fn  # ✓ IMPORTANT!
)

print(f'✓ Train batches: {len(train_loader)}, Val batches: {len(val_loader)}')

# Test batch
print('\nTesting data loading...')
test_batch = next(iter(train_loader))
print(f'✓ Batch image shape: {test_batch["image"].shape}')
print(f'✓ Targets structure: {len(test_batch["targets"]["cls_labels"])} FPN levels')

# START TRAINING
print('\n' + '='*80)
print('🚀 STARTING TRAINING - ALL BUGS FIXED!')
print('='*80 + '\n')

trained_model = train_model(
    model, 
    train_loader, 
    val_loader, 
    num_epochs=NUM_EPOCHS, 
    device=DEVICE,
    resume_from=None,
    save_dir=SAVE_DIR
)

print('\n✅ TRAINING COMPLETE!')

✓ Data directory: /kaggle/input/webface-112x112/webface_112x112

Loading dataset...
Loading dataset from: /kaggle/input/webface-112x112/webface_112x112
✓ Found 490623 images
✓ Train: 441560, Val: 49063
✓ Train batches: 13798, Val batches: 1534

Testing data loading...
✓ Batch image shape: torch.Size([32, 3, 224, 224])
✓ Targets structure: 4 FPN levels

🚀 STARTING TRAINING - ALL BUGS FIXED!


Starting training from epoch 0


Epoch 1/50
------------------------------------------------------------
Epoch 1, Batch 0/13798, Loss: 2.4778, Cls: 0.0432, Bbox: 0.2113
Epoch 1, Batch 100/13798, Loss: 0.0203, Cls: 0.0023, Bbox: 0.0008
Epoch 1, Batch 200/13798, Loss: 0.0131, Cls: 0.0016, Bbox: 0.0006
Epoch 1, Batch 300/13798, Loss: 0.0170, Cls: 0.0012, Bbox: 0.0012
Epoch 1, Batch 400/13798, Loss: 0.0096, Cls: 0.0012, Bbox: 0.0005
Epoch 1, Batch 500/13798, Loss: 0.0044, Cls: 0.0007, Bbox: 0.0001
Epoch 1, Batch 600/13798, Loss: 0.0058, Cls: 0.0005, Bbox: 0.0004
Epoch 1, Batch 700/13798, Loss: 0.0055, 

KeyboardInterrupt: 

In [45]:
# ============================================================================
# COMPLETE TRAINING SCRIPT
# ============================================================================
import os
import torch
from torch.utils.data import DataLoader, Subset
import numpy as np

# --- 1. SETUP & CONFIGURATION ---
# Auto-detect data directory
BASE_DIR = '/kaggle/input/webface-112x112'

# Verify directory exists before proceeding
if os.path.exists(BASE_DIR):
    subdirs = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
    # Handle case where dataset might be directly in BASE_DIR or a subdir
    if subdirs:
        DATA_DIR = os.path.join(BASE_DIR, subdirs[0])
    else:
        DATA_DIR = BASE_DIR
else:
    # Fallback or raise error if path is wrong
    DATA_DIR = BASE_DIR 
    print(f"Warning: Base directory {BASE_DIR} not found. Check paths.")

print(f'✓ Data directory: {DATA_DIR}')

# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 50
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SAVE_DIR = './checkpoints'
RESUME_FROM = None  # Example: './checkpoints/last_checkpoint.pth'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

print(f'✓ Device: {DEVICE}')
print(f'✓ Batch size: {BATCH_SIZE}')
print(f'✓ Epochs: {NUM_EPOCHS}')

# --- 2. LOAD DATASET ---
print('\nLoading dataset...')

# NOTE: Ensure 'WebFaceDataset' class is defined in a previous cell!
try:
    full_dataset = WebFaceDataset(DATA_DIR, augment=True, img_size=224)
except NameError:
    print("❌ Error: 'WebFaceDataset' is not defined. Please run the cell defining the dataset class first.")
    raise

total_size = len(full_dataset)
train_size = int(0.9 * total_size)
val_size = total_size - train_size

indices = np.arange(total_size)
np.random.shuffle(indices)

train_dataset = Subset(full_dataset, indices[:train_size])
val_dataset = Subset(full_dataset, indices[train_size:])

print(f'✓ Train: {len(train_dataset)}, Val: {len(val_dataset)}')

# --- 3. DATALOADERS ---
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2, 
    pin_memory=True, 
    drop_last=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=2, 
    pin_memory=True
)

print(f'✓ Train batches: {len(train_loader)}, Val batches: {len(val_loader)}')

# --- 4. SANITY CHECK ---
print('\nTesting data loading...')
try:
    test_batch = next(iter(train_loader))
    print(f'✓ Batch image shape: {test_batch["image"].shape}')
    print(f'✓ Data loading successful!')
except Exception as e:
    print(f"❌ Error loading batch: {e}")
    raise

# --- 5. TRAINING LOOP ---
print('\n' + '='*80)
print('🚀 STARTING TRAINING')
print('='*80 + '\n')

# NOTE: Ensure 'model' and 'train_model' are defined in previous cells!
try:
    trained_model = train_model(
        model, 
        train_loader, 
        val_loader, 
        num_epochs=NUM_EPOCHS, 
        device=DEVICE,
        resume_from=RESUME_FROM,
        save_dir=SAVE_DIR
    )
    
    print('\n' + '='*80)
    print('✅ TRAINING COMPLETE!')
    print('='*80)

except NameError as e:
    print(f"❌ Error: Missing definitions. {e}")
    print("Ensure 'model' and 'train_model' function are defined and run before this block.")

✓ Data directory: /kaggle/input/webface-112x112/webface_112x112
✓ Device: cuda
✓ Batch size: 32
✓ Epochs: 50

Loading dataset...
Loading dataset from: /kaggle/input/webface-112x112/webface_112x112
✓ Found 490623 images
✓ Train: 441560, Val: 49063
✓ Train batches: 13798, Val batches: 1534

Testing data loading...
✓ Batch image shape: torch.Size([32, 3, 224, 224])
✓ Data loading successful!

🚀 STARTING TRAINING


Starting training from epoch 0


Epoch 1/50
------------------------------------------------------------


ValueError: Target size (torch.Size([1204224, 2])) must be the same as input size (torch.Size([301056, 2]))

In [40]:
import os
import torch
from torch.utils.data import DataLoader, Subset
import numpy as np

# Auto-detect correct directory
BASE_DIR = '/kaggle/input/webface-112x112'
subdirs = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
DATA_DIR = os.path.join(BASE_DIR, subdirs[0])

print(f'✓ Using data directory: {DATA_DIR}')

BATCH_SIZE = 32
NUM_EPOCHS = 50
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'✓ Device: {DEVICE}')

# Create dataset
print('\nLoading dataset...')
full_dataset = WebFaceDataset(DATA_DIR, augment=True, img_size=224)

print(f'✓ Total images: {len(full_dataset)}')

# Train/Val split
total_size = len(full_dataset)
train_size = int(0.9 * total_size)
val_size = total_size - train_size

indices = np.arange(total_size)
np.random.shuffle(indices)

train_dataset = Subset(full_dataset, indices[:train_size])
val_dataset = Subset(full_dataset, indices[train_size:])

print(f'✓ Train: {len(train_dataset)}, Val: {len(val_dataset)}')

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                         num_workers=2, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                       num_workers=2, pin_memory=True)

print(f'✓ Train batches: {len(train_loader)}, Val batches: {len(val_loader)}')

# Test batch
test_batch = next(iter(train_loader))
print(f'✓ Batch shape: {test_batch["image"].shape}')

# Train
print('\n🚀 Starting training...')
trained_model = train_model(model, train_loader, val_loader, num_epochs=NUM_EPOCHS, device=DEVICE)

✓ Using data directory: /kaggle/input/webface-112x112/webface_112x112
✓ Device: cuda

Loading dataset...
Loading dataset from: /kaggle/input/webface-112x112/webface_112x112
✓ Found 490623 images


TypeError: object of type 'WebFaceDataset' has no len()